***GENERATED CODE FOR classify2 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Order ID', 'transformation_label': 'String Indexer'}], 'feature': 'Order ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '79', 'mean': '', 'stddev': '', 'min': 'CA-2012-AB10015140-40958', 'max': 'US-2015-NP1832582-42216', 'missing': '0', 'distinct': '71'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Order ID'}, {'feature_label': 'Order ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Order ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Order Date', 'transformation_label': 'String Indexer'}], 'feature': 'Order Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '79', 'mean': '', 'stddev': '', 'min': '1/10/2012', 'max': '9/8/2015', 'missing': '0', 'distinct': '69'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Order Date'}, {'feature_label': 'Order Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Order Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Customer ID', 'transformation_label': 'String Indexer'}], 'feature': 'Customer ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '79', 'mean': '', 'stddev': '', 'min': 'AB-100151402', 'max': 'ZC-2191048', 'missing': '0', 'distinct': '66'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Customer ID'}, {'feature_label': 'Customer ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Customer ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Customer Name', 'transformation_label': 'String Indexer'}], 'feature': 'Customer Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '79', 'mean': '', 'stddev': '', 'min': 'Aaron Bergman', 'max': 'Zuschuss Carroll', 'missing': '0', 'distinct': '60'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Customer Name'}, {'feature_label': 'Customer Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Customer Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Segment', 'transformation_label': 'String Indexer'}], 'feature': 'Segment', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '79', 'mean': '', 'stddev': '', 'min': 'Consumer', 'max': 'Home Office', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Segment'}, {'feature_label': 'Segment', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Segment')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'City', 'transformation_label': 'String Indexer'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '79', 'mean': '', 'stddev': '', 'min': 'Arlington', 'max': 'Wuxi', 'missing': '0', 'distinct': '68'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'feature_label': 'City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '79', 'mean': '', 'stddev': '', 'min': 'Andalus�a', 'max': 'Western Australia', 'missing': '0', 'distinct': '57'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Country', 'transformation_label': 'String Indexer'}], 'feature': 'Country', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '79', 'mean': '', 'stddev': '', 'min': 'Afghanistan', 'max': 'Uruguay', 'missing': '0', 'distinct': '28'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Country'}, {'feature_label': 'Country', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Country')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Region', 'transformation_label': 'String Indexer'}], 'feature': 'Region', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '79', 'mean': '', 'stddev': '', 'min': 'Caribbean', 'max': 'Western US', 'missing': '0', 'distinct': '19'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Region'}, {'feature_label': 'Region', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Region')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Market', 'transformation_label': 'String Indexer'}], 'feature': 'Market', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '79', 'mean': '', 'stddev': '', 'min': 'Africa', 'max': 'USA', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Market'}, {'feature_label': 'Market', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Market')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Category', 'transformation_label': 'String Indexer'}], 'feature': 'Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '79', 'mean': '', 'stddev': '', 'min': 'Furniture', 'max': 'Technology', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Category'}, {'feature_label': 'Category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Sub-Category', 'transformation_label': 'String Indexer'}], 'feature': 'Sub-Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '79', 'mean': '', 'stddev': '', 'min': 'Accessories', 'max': 'Tables', 'missing': '0', 'distinct': '15'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Sub-Category'}, {'feature_label': 'Sub-Category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Sub-Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Product Name', 'transformation_label': 'String Indexer'}], 'feature': 'Product Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '79', 'mean': '', 'stddev': '', 'min': '"Carina 42""Hx23 3/4""W Media Storage Unit"', 'max': 'Verbatim 25 GB 6x Blu-ray Single Layer Recordable Disc, 10/Pack', 'missing': '0', 'distinct': '69'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product Name'}, {'feature_label': 'Product Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Product Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Order Priority', 'transformation_label': 'String Indexer'}], 'feature': 'Order Priority', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '79', 'mean': '', 'stddev': '', 'min': 'Critical', 'max': 'Medium', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Order Priority'}, {'feature_label': 'Order Priority', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Order Priority')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run classify2Hooks.ipynb
try:
	#sourcePreExecutionHook()

	globalorderssuperstore = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/globalorders_superstore_2016.csv', 'filename': 'globalorders_superstore_2016.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(globalorderssuperstore)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run classify2Hooks.ipynb
try:
	#transformationPreExecutionHook()

	classifyautofe = TransformationMain.run(globalorderssuperstore,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Order ID", "transformation_label": "String Indexer"}], "feature": "Order ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "79", "mean": "", "stddev": "", "min": "CA-2012-AB10015140-40958", "max": "US-2015-NP1832582-42216", "missing": "0", "distinct": "71"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Order ID"}, {"transformationsData": [{"feature_label": "Order Date", "transformation_label": "String Indexer"}], "feature": "Order Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "79", "mean": "", "stddev": "", "min": "1/10/2012", "max": "9/8/2015", "missing": "0", "distinct": "69"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Order Date"}, {"transformationsData": [{"feature_label": "Customer ID", "transformation_label": "String Indexer"}], "feature": "Customer ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "79", "mean": "", "stddev": "", "min": "AB-100151402", "max": "ZC-2191048", "missing": "0", "distinct": "66"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer ID"}, {"transformationsData": [{"feature_label": "Customer Name", "transformation_label": "String Indexer"}], "feature": "Customer Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "79", "mean": "", "stddev": "", "min": "Aaron Bergman", "max": "Zuschuss Carroll", "missing": "0", "distinct": "60"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer Name"}, {"transformationsData": [{"feature_label": "Segment", "transformation_label": "String Indexer"}], "feature": "Segment", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "79", "mean": "", "stddev": "", "min": "Consumer", "max": "Home Office", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Segment"}, {"transformationsData": [{"feature_label": "City", "transformation_label": "String Indexer"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "79", "mean": "", "stddev": "", "min": "Arlington", "max": "Wuxi", "missing": "0", "distinct": "68"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "79", "mean": "", "stddev": "", "min": "Andalus\ufffda", "max": "Western Australia", "missing": "0", "distinct": "57"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"feature_label": "Country", "transformation_label": "String Indexer"}], "feature": "Country", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "79", "mean": "", "stddev": "", "min": "Afghanistan", "max": "Uruguay", "missing": "0", "distinct": "28"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Country"}, {"transformationsData": [{"feature_label": "Region", "transformation_label": "String Indexer"}], "feature": "Region", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "79", "mean": "", "stddev": "", "min": "Caribbean", "max": "Western US", "missing": "0", "distinct": "19"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Region"}, {"transformationsData": [{"feature_label": "Market", "transformation_label": "String Indexer"}], "feature": "Market", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "79", "mean": "", "stddev": "", "min": "Africa", "max": "USA", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Market"}, {"transformationsData": [{"feature_label": "Category", "transformation_label": "String Indexer"}], "feature": "Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "79", "mean": "", "stddev": "", "min": "Furniture", "max": "Technology", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Category"}, {"transformationsData": [{"feature_label": "Sub-Category", "transformation_label": "String Indexer"}], "feature": "Sub-Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "79", "mean": "", "stddev": "", "min": "Accessories", "max": "Tables", "missing": "0", "distinct": "15"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Sub-Category"}, {"transformationsData": [{"feature_label": "Product Name", "transformation_label": "String Indexer"}], "feature": "Product Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "79", "mean": "", "stddev": "", "min": "\"Carina 42\"\"Hx23 3/4\"\"W Media Storage Unit\"", "max": "Verbatim 25 GB 6x Blu-ray Single Layer Recordable Disc, 10/Pack", "missing": "0", "distinct": "69"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product Name"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Sales", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "79", "mean": "2392.37", "stddev": "1669.65", "min": "8.0", "max": "7958.58", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Sales"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Quantity", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "79", "mean": "6.14", "stddev": "3.0", "min": "1", "max": "14", "missing": "0"}, "updatedLabel": "Quantity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Discount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "79", "mean": "0.06", "stddev": "0.11", "min": "0.0", "max": "0.5", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Discount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Profit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "79", "mean": "509.97", "stddev": "645.39", "min": "-452.81", "max": "3979.08", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Profit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Shipping Cost", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "79", "mean": "534.76", "stddev": "302.14", "min": "1.13", "max": "923.63", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Shipping Cost"}, {"transformationsData": [{"feature_label": "Order Priority", "transformation_label": "String Indexer"}], "feature": "Order Priority", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "79", "mean": "", "stddev": "", "min": "Critical", "max": "Medium", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Order Priority"}]}))

	#transformationPostExecutionHook(classifyautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run classify2Hooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(classifyautofe, ["Sales", "Quantity", "Discount", "Profit", "Shipping Cost", "Order ID_stringindexer", "Order Date_stringindexer", "Customer ID_stringindexer", "Customer Name_stringindexer", "Segment_stringindexer", "City_stringindexer", "State_stringindexer", "Country_stringindexer", "Market_stringindexer", "Category_stringindexer", "Sub-Category_stringindexer", "Product Name_stringindexer", "Order Priority_stringindexer"], "Region_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

